In [ ]:
pip install yfinance


In [ ]:
# Step 1: Install required packages
!pip install yfinance plotly scikit-learn pandas

# Step 2: Import necessary libraries
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest

# Step 3: Fetch Market Data using Yahoo Finance API
STOCK_SYMBOL = 'AAPL'  # Apple Inc. stock symbol (you can change this to any valid stock symbol)
DATA_PERIOD = '1y'  # Fetching 1 year of data

# Download stock data
stock_data = yf.download(STOCK_SYMBOL, period=DATA_PERIOD)

# Save the data to a CSV file in Colab's environment
csv_filename = f'{STOCK_SYMBOL}_yfinance_data.csv'
stock_data.to_csv(csv_filename)

print(f"Market data for {STOCK_SYMBOL} saved to {csv_filename}")

# Step 4: Load Data from the CSV File and Preprocess
# Load the CSV data into a pandas DataFrame
df = pd.read_csv(csv_filename)

# Convert 'Date' column to datetime format and set as index
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# For simplicity, we are using the 'Close' price for anomaly detection
market_data = df[['Close']].copy()

# Generate rolling features (for example, rolling mean and rolling std)
market_data['Rolling_Mean'] = market_data['Close'].rolling(window=5).mean()
market_data['Rolling_Std'] = market_data['Close'].rolling(window=5).std()

# Drop NaN values from rolling calculations
market_data.dropna(inplace=True)

# Step 5: Apply Isolation Forest for Anomaly Detection
# Prepare features for the Isolation Forest model
X = market_data[['Close', 'Rolling_Mean', 'Rolling_Std']].values

# Train Isolation Forest model
iso_forest = IsolationForest(contamination=0.01, random_state=42)
market_data['Anomaly'] = iso_forest.fit_predict(X)

# Mark anomalies (-1 indicates anomaly, 1 indicates normal)
market_data['Anomaly'] = market_data['Anomaly'].apply(lambda x: 1 if x == -1 else 0)

print("Anomaly detection completed.")

# Step 6: Visualize Market Data and Anomalies using Plotly
def create_dashboard(data):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.1, subplot_titles=('Market Price', 'Anomalies'))

    # Market Price Plot
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Market Price'), row=1, col=1)

    # Anomaly Plot
    anomalies = data[data['Anomaly'] == 1]
    fig.add_trace(go.Scatter(x=anomalies.index, y=anomalies['Close'], mode='markers',
                             marker=dict(color='red', size=8), name='Anomalies'), row=1, col=1)

    # Adding rolling mean and standard deviation to visualize market patterns
    fig.add_trace(go.Scatter(x=data.index, y=data['Rolling_Mean'], mode='lines',
                             name='Rolling Mean'), row=2, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Rolling_Std'], mode='lines',
                             name='Rolling Std'), row=2, col=1)

    fig.update_layout(title_text="Market Data and Anomalies", height=700)
    fig.show()

# Plot the dashboard
create_dashboard(market_data)


[*********************100%***********************]  1 of 1 completed


Market data for AAPL saved to AAPL_yfinance_data.csv
Anomaly detection completed.


In [ ]:
# Step 1: Install necessary libraries
!pip install yfinance plotly scikit-learn pandas statsmodels

# Step 2: Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.arima.model import ARIMA

# Step 3: Fetch Market Data and Currency Data using Yahoo Finance
STOCK_SYMBOL = 'AAPL'  # Stock symbol for region risk
CURRENCY_PAIR = 'EURUSD=X'  # Currency pair for currency risk
DATA_PERIOD = '1y'  # 1 year of data

# Download stock data for market risk
stock_data = yf.download(STOCK_SYMBOL, period=DATA_PERIOD)
# Download currency data for currency risk
currency_data = yf.download(CURRENCY_PAIR, period=DATA_PERIOD)

# Save the data to CSVs
stock_data.to_csv(f'{STOCK_SYMBOL}_market_data.csv')
currency_data.to_csv(f'{CURRENCY_PAIR}_currency_data.csv')

print(f"Market data for {STOCK_SYMBOL} and currency data for {CURRENCY_PAIR} saved to CSV files.")

# Step 4: Data Preprocessing
# Preprocess stock data
stock_data['Date'] = pd.to_datetime(stock_data.index)
stock_data.set_index('Date', inplace=True)
stock_data = stock_data[['Close']].dropna()

# Preprocess currency data
currency_data['Date'] = pd.to_datetime(currency_data.index)
currency_data.set_index('Date', inplace=True)
currency_data = currency_data[['Close']].dropna()

# Step 5: Region Risk (Anomaly Detection in Market Data)
# Generate rolling statistics for stock data
stock_data['Rolling_Mean'] = stock_data['Close'].rolling(window=5).mean()
stock_data['Rolling_Std'] = stock_data['Close'].rolling(window=5).std()
stock_data.dropna(inplace=True)

# Train IsolationForest model for anomaly detection
iso_forest = IsolationForest(contamination=0.01, random_state=42)
stock_data['Anomaly'] = iso_forest.fit_predict(stock_data[['Close', 'Rolling_Mean', 'Rolling_Std']])

# Mark anomalies (1 indicates anomaly, 0 indicates normal)
stock_data['Anomaly'] = stock_data['Anomaly'].apply(lambda x: 1 if x == -1 else 0)

print("Anomaly detection in market data completed.")

# Step 6: Currency Risk (Prediction and Hedging Strategy)
# ARIMA model for predicting currency movements
# Fit ARIMA(5,1,0) model on currency data (for simplicity, you can experiment with different parameters)
arima_model = ARIMA(currency_data['Close'], order=(5, 1, 0))
arima_fit = arima_model.fit()

# Forecast next 10 days of currency movements
forecast_steps = 10
currency_forecast = arima_fit.forecast(steps=forecast_steps)
forecast_dates = pd.date_range(currency_data.index[-1], periods=forecast_steps + 1, freq='D')[1:]

# Create a DataFrame for forecasted currency prices
currency_forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecasted_Price': currency_forecast})
currency_forecast_df.set_index('Date', inplace=True)

# Hedging Strategy Simulation: Simple Example (This can be expanded)
def hedging_strategy(predicted_prices, actual_prices, threshold=0.01):
    """Simulates a simple hedging strategy by comparing predicted vs. actual prices"""
    hedging_decisions = []
    for pred, actual in zip(predicted_prices, actual_prices):
        if abs(pred - actual) / actual > threshold:  # Significant fluctuation
            hedging_decisions.append('Hedge')  # Hedge when prediction differs significantly
        else:
            hedging_decisions.append('No Hedge')
    return hedging_decisions

# Apply hedging strategy on forecasted vs. actual prices
hedging_decisions = hedging_strategy(currency_forecast, currency_data['Close'][-forecast_steps:])

# Step 7: Visualize Market Data Anomalies and Currency Forecast with Dashboard
def create_dashboard(stock_data, currency_data, currency_forecast_df, hedging_decisions):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=('Market Price with Anomalies', 'Currency Price & Forecast', 'Hedging Decisions'))

    # Plot 1: Market Price and Anomalies
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Market Price'), row=1, col=1)
    anomalies = stock_data[stock_data['Anomaly'] == 1]
    fig.add_trace(go.Scatter(x=anomalies.index, y=anomalies['Close'], mode='markers',
                             marker=dict(color='red', size=8), name='Anomalies'), row=1, col=1)

    # Plot 2: Currency Prices and Forecast
    fig.add_trace(go.Scatter(x=currency_data.index, y=currency_data['Close'], mode='lines', name='Currency Price (Actual)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=currency_forecast_df.index, y=currency_forecast_df['Forecasted_Price'],
                             mode='lines', name='Currency Price (Forecast)'), row=2, col=1)

    # Plot 3: Hedging Decisions
    decision_dates = currency_data.index[-forecast_steps:]
    fig.add_trace(go.Scatter(x=decision_dates, y=currency_data['Close'][-forecast_steps:], mode='lines+markers',
                             marker=dict(color=['green' if h == 'No Hedge' else 'red' for h in hedging_decisions], size=8),
                             name='Hedging Decision'), row=3, col=1)

    fig.update_layout(height=900, title_text="Market Data Risk and Currency Risk Dashboard")
    fig.show()

# Display the dashboard
create_dashboard(stock_data, currency_data, currency_forecast_df, hedging_decisions)

# Step 8: Alert System for Currency Fluctuations
def currency_alert_system(currency_data, forecast, threshold=0.02):
    alerts = []
    forecast_steps = len(forecast)

    # Loop through the forecasted data and the actual data (last 'forecast_steps' rows)
    for i, (actual, predicted) in enumerate(zip(currency_data['Close'][-forecast_steps:], forecast)):
        if abs(actual - predicted) / actual > threshold:
            alert_date = currency_data.index[-forecast_steps:][i].date()  # Get the date from the DataFrame
            alerts.append(f"Significant currency movement detected on {alert_date}.")

    return alerts


# Generate alerts for significant currency movements
alerts = currency_alert_system(currency_data, currency_forecast)
if alerts:
    print("Currency Alerts:")
    for alert in alerts:
        print(alert)
else:
    print("No significant currency movements detected.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Market data for AAPL and currency data for EURUSD=X saved to CSV files.
Anomaly detection in market data completed.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the 

No significant currency movements detected.


In [ ]:
# Step 1: Install necessary libraries
!pip install yfinance plotly scikit-learn pandas statsmodels

# Step 2: Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.arima.model import ARIMA

# Step 3: Fetch Market Data and Currency Data using Yahoo Finance
STOCK_SYMBOL = 'AAPL'  # Stock symbol for region risk
CURRENCY_PAIR = 'EURUSD=X'  # Currency pair for currency risk
DATA_PERIOD = '1y'  # 1 year of data

# Download stock data for market risk
stock_data = yf.download(STOCK_SYMBOL, period=DATA_PERIOD)
# Download currency data for currency risk
currency_data = yf.download(CURRENCY_PAIR, period=DATA_PERIOD)

# Save the data to CSVs
stock_data.to_csv(f'{STOCK_SYMBOL}_market_data.csv')
currency_data.to_csv(f'{CURRENCY_PAIR}_currency_data.csv')

print(f"Market data for {STOCK_SYMBOL} and currency data for {CURRENCY_PAIR} saved to CSV files.")

# Step 4: Data Preprocessing
# Preprocess stock data
stock_data['Date'] = pd.to_datetime(stock_data.index)
stock_data.set_index('Date', inplace=True)
stock_data = stock_data[['Close']].dropna()

# Preprocess currency data
currency_data['Date'] = pd.to_datetime(currency_data.index)
currency_data.set_index('Date', inplace=True)
currency_data = currency_data[['Close']].dropna()

# Step 5: Region Risk (Anomaly Detection in Market Data)
# Generate rolling statistics for stock data
stock_data['Rolling_Mean'] = stock_data['Close'].rolling(window=5).mean()
stock_data['Rolling_Std'] = stock_data['Close'].rolling(window=5).std()
stock_data.dropna(inplace=True)

# Train IsolationForest model for anomaly detection
iso_forest = IsolationForest(contamination=0.01, random_state=42)
stock_data['Anomaly'] = iso_forest.fit_predict(stock_data[['Close', 'Rolling_Mean', 'Rolling_Std']])

# Mark anomalies (1 indicates anomaly, 0 indicates normal)
stock_data['Anomaly'] = stock_data['Anomaly'].apply(lambda x: 1 if x == -1 else 0)

print("Anomaly detection in market data completed.")

# Step 6: Currency Risk (Prediction and Hedging Strategy)
# ARIMA model for predicting currency movements
# Fit ARIMA(5,1,0) model on currency data (for simplicity, you can experiment with different parameters)
arima_model = ARIMA(currency_data['Close'], order=(5, 1, 0))
arima_fit = arima_model.fit()

# Forecast next 10 days of currency movements
forecast_steps = 10
currency_forecast = arima_fit.forecast(steps=forecast_steps)
forecast_dates = pd.date_range(currency_data.index[-1], periods=forecast_steps + 1, freq='D')[1:]

# Create a DataFrame for forecasted currency prices
currency_forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecasted_Price': currency_forecast})
currency_forecast_df.set_index('Date', inplace=True)

# Hedging Strategy Simulation: Simple Example (This can be expanded)
def hedging_strategy(predicted_prices, actual_prices, threshold=0.01):
    """Simulates a simple hedging strategy by comparing predicted vs. actual prices"""
    hedging_decisions = []
    for pred, actual in zip(predicted_prices, actual_prices):
        if abs(pred - actual) / actual > threshold:  # Significant fluctuation
            hedging_decisions.append('Hedge')  # Hedge when prediction differs significantly
        else:
            hedging_decisions.append('No Hedge')
    return hedging_decisions

# Apply hedging strategy on forecasted vs. actual prices
hedging_decisions = hedging_strategy(currency_forecast, currency_data['Close'][-forecast_steps:])

# Step 7: Visualize Market Data Anomalies and Currency Forecast with Dashboard
def create_dashboard(stock_data, currency_data, currency_forecast_df, hedging_decisions):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=('Market Price with Anomalies', 'Currency Price & Forecast', 'Hedging Decisions'))

    # Plot 1: Market Price and Anomalies
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Market Price'), row=1, col=1)
    anomalies = stock_data[stock_data['Anomaly'] == 1]
    fig.add_trace(go.Scatter(x=anomalies.index, y=anomalies['Close'], mode='markers',
                             marker=dict(color='red', size=8), name='Anomalies'), row=1, col=1)

    # Plot 2: Currency Prices and Forecast
    fig.add_trace(go.Scatter(x=currency_data.index, y=currency_data['Close'], mode='lines', name='Currency Price (Actual)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=currency_forecast_df.index, y=currency_forecast_df['Forecasted_Price'],
                             mode='lines', name='Currency Price (Forecast)'), row=2, col=1)

    # Plot 3: Hedging Decisions
    decision_dates = currency_data.index[-forecast_steps:]
    fig.add_trace(go.Scatter(x=decision_dates, y=currency_data['Close'][-forecast_steps:], mode='lines+markers',
                             marker=dict(color=['green' if h == 'No Hedge' else 'red' for h in hedging_decisions], size=8),
                             name='Hedging Decision'), row=3, col=1)

    fig.update_layout(height=900, title_text="Market Data Risk and Currency Risk Dashboard")
    fig.show()

# Display the dashboard
create_dashboard(stock_data, currency_data, currency_forecast_df, hedging_decisions)

# Step 8: Alert System for Currency Fluctuations
def currency_alert_system(currency_data, forecast, threshold=0.02):
    alerts = []
    forecast_steps = len(forecast)

    # Loop through the forecasted data and the actual data (last 'forecast_steps' rows)
    for i, (actual, predicted) in enumerate(zip(currency_data['Close'][-forecast_steps:], forecast)):
        if abs(actual - predicted) / actual > threshold:
            alert_date = currency_data.index[-forecast_steps:][i].date()  # Get the date from the DataFrame
            alerts.append(f"Significant currency movement detected on {alert_date}.")

    return alerts


# Generate alerts for significant currency movements
alerts = currency_alert_system(currency_data, currency_forecast)
if alerts:
    print("Currency Alerts:")
    for alert in alerts:
        print(alert)
else:
    print("No significant currency movements detected.")


In [ ]:
# Step 1: Install necessary libraries
!pip install yfinance plotly scikit-learn pandas statsmodels

# Step 2: Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.arima.model import ARIMA

# Step 3: Fetch Market Data and Currency Data using Yahoo Finance
STOCK_SYMBOL = 'AAPL'  # Stock symbol for region risk
CURRENCY_PAIR = 'EURUSD=X'  # Currency pair for currency risk
DATA_PERIOD = '1y'  # 1 year of data

# Download stock data for market risk
stock_data = yf.download(STOCK_SYMBOL, period=DATA_PERIOD)
# Download currency data for currency risk
currency_data = yf.download(CURRENCY_PAIR, period=DATA_PERIOD)

# Save the data to CSVs
stock_data.to_csv(f'{STOCK_SYMBOL}_market_data.csv')
currency_data.to_csv(f'{CURRENCY_PAIR}_currency_data.csv')

print(f"Market data for {STOCK_SYMBOL} and currency data for {CURRENCY_PAIR} saved to CSV files.")

# Step 4: Data Preprocessing
# Preprocess stock data
stock_data['Date'] = pd.to_datetime(stock_data.index)
stock_data.set_index('Date', inplace=True)
stock_data = stock_data[['Close']].dropna()

# Preprocess currency data
currency_data['Date'] = pd.to_datetime(currency_data.index)
currency_data.set_index('Date', inplace=True)
currency_data = currency_data[['Close']].dropna()

# Step 5: Region Risk (Anomaly Detection in Market Data)
# Generate rolling statistics for stock data
stock_data['Rolling_Mean'] = stock_data['Close'].rolling(window=5).mean()
stock_data['Rolling_Std'] = stock_data['Close'].rolling(window=5).std()
stock_data.dropna(inplace=True)

# Train IsolationForest model for anomaly detection
iso_forest = IsolationForest(contamination=0.01, random_state=42)
stock_data['Anomaly'] = iso_forest.fit_predict(stock_data[['Close', 'Rolling_Mean', 'Rolling_Std']])

# Mark anomalies (1 indicates anomaly, 0 indicates normal)
stock_data['Anomaly'] = stock_data['Anomaly'].apply(lambda x: 1 if x == -1 else 0)

print("Anomaly detection in market data completed.")

# Step 6: Currency Risk (Prediction and Hedging Strategy)
# ARIMA model for predicting currency movements
# Fit ARIMA(5,1,0) model on currency data (for simplicity, you can experiment with different parameters)
arima_model = ARIMA(currency_data['Close'], order=(5, 1, 0))
arima_fit = arima_model.fit()

# Forecast next 10 days of currency movements
forecast_steps = 10
currency_forecast = arima_fit.forecast(steps=forecast_steps)
forecast_dates = pd.date_range(currency_data.index[-1], periods=forecast_steps + 1, freq='D')[1:]

# Create a DataFrame for forecasted currency prices
currency_forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecasted_Price': currency_forecast})
currency_forecast_df.set_index('Date', inplace=True)

# Hedging Strategy Simulation: Simple Example (This can be expanded)
def hedging_strategy(predicted_prices, actual_prices, threshold=0.01):
    """Simulates a simple hedging strategy by comparing predicted vs. actual prices"""
    hedging_decisions = []
    for pred, actual in zip(predicted_prices, actual_prices):
        if abs(pred - actual) / actual > threshold:  # Significant fluctuation
            hedging_decisions.append('Hedge')  # Hedge when prediction differs significantly
        else:
            hedging_decisions.append('No Hedge')
    return hedging_decisions

# Apply hedging strategy on forecasted vs. actual prices
hedging_decisions = hedging_strategy(currency_forecast, currency_data['Close'][-forecast_steps:])

# Step 7: Visualize Market Data Anomalies and Currency Forecast with Dashboard
def create_dashboard(stock_data, currency_data, currency_forecast_df, hedging_decisions):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=('Market Price with Anomalies', 'Currency Price & Forecast', 'Hedging Decisions'))

    # Plot 1: Market Price and Anomalies
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Market Price'), row=1, col=1)
    anomalies = stock_data[stock_data['Anomaly'] == 1]
    fig.add_trace(go.Scatter(x=anomalies.index, y=anomalies['Close'], mode='markers',
                             marker=dict(color='red', size=8), name='Anomalies'), row=1, col=1)

    # Plot 2: Currency Prices and Forecast
    fig.add_trace(go.Scatter(x=currency_data.index, y=currency_data['Close'], mode='lines', name='Currency Price (Actual)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=currency_forecast_df.index, y=currency_forecast_df['Forecasted_Price'],
                             mode='lines', name='Currency Price (Forecast)'), row=2, col=1)

    # Plot 3: Hedging Decisions
    decision_dates = currency_data.index[-forecast_steps:]
    fig.add_trace(go.Scatter(x=decision_dates, y=currency_data['Close'][-forecast_steps:], mode='lines+markers',
                             marker=dict(color=['green' if h == 'No Hedge' else 'red' for h in hedging_decisions], size=8),
                             name='Hedging Decision'), row=3, col=1)

    fig.update_layout(height=900, title_text="Market Data Risk and Currency Risk Dashboard")
    fig.show()

# Display the dashboard
create_dashboard(stock_data, currency_data, currency_forecast_df, hedging_decisions)

# Step 8: Alert System for Currency Fluctuations
def currency_alert_system(currency_data, forecast, threshold=0.02):
    alerts = []
    forecast_steps = len(forecast)

    # Loop through the forecasted data and the actual data (last 'forecast_steps' rows)
    for i, (actual, predicted) in enumerate(zip(currency_data['Close'][-forecast_steps:], forecast)):
        if abs(actual - predicted) / actual > threshold:
            alert_date = currency_data.index[-forecast_steps:][i].date()  # Get the date from the DataFrame
            alerts.append(f"Significant currency movement detected on {alert_date}.")

    return alerts


# Generate alerts for significant currency movements
alerts = currency_alert_system(currency_data, currency_forecast)
if alerts:
    print("Currency Alerts:")
    for alert in alerts:
        print(alert)
else:
    print("No significant currency movements detected.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.



Market data for AAPL and currency data for EURUSD=X saved to CSV files.
Anomaly detection in market data completed.


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



No significant currency movements detected.


In [ ]:
!pip install yfinance plotly scikit-learn pandas statsmodels Flask


In [ ]:
# Step 1: Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.ensemble import IsolationForest
from statsmodels.tsa.arima.model import ARIMA
from flask import Flask, render_template, request

# Step 2: Preprocessing with Date Frequency
def preprocess_data(data, col_name='Close'):
    # Set date index frequency to 'D' (daily)
    data['Date'] = pd.to_datetime(data.index)
    data.set_index('Date', inplace=True)
    data = data.asfreq('D')  # Ensure date frequency
    data = data[[col_name]].dropna()
    return data

# Step 3: Function to fetch and process data based on stock symbol
def fetch_market_data(stock_symbol='AAPL', currency_pair='EURUSD=X', data_period='1y'):
    # Download stock and currency data
    stock_data = yf.download(stock_symbol, period=data_period)
    currency_data = yf.download(currency_pair, period=data_period)

    # Preprocess data with frequency handling
    stock_data = preprocess_data(stock_data)
    currency_data = preprocess_data(currency_data)

    return stock_data, currency_data

# Step 4: Anomaly Detection using Isolation Forest
def detect_anomalies(stock_data):
    stock_data['Rolling_Mean'] = stock_data['Close'].rolling(window=5).mean()
    stock_data['Rolling_Std'] = stock_data['Close'].rolling(window=5).std()
    stock_data.dropna(inplace=True)

    iso_forest = IsolationForest(contamination=0.01, random_state=42)
    stock_data['Anomaly'] = iso_forest.fit_predict(stock_data[['Close', 'Rolling_Mean', 'Rolling_Std']])
    stock_data['Anomaly'] = stock_data['Anomaly'].apply(lambda x: 1 if x == -1 else 0)

    return stock_data

# Step 5: Currency Forecast with ARIMA
def currency_forecast(currency_data, forecast_steps=10):
    # Fit ARIMA model with frequency-corrected data
    arima_model = ARIMA(currency_data['Close'], order=(5, 1, 0))
    arima_fit = arima_model.fit()

    # Forecast currency movements
    currency_forecast = arima_fit.forecast(steps=forecast_steps)
    forecast_dates = pd.date_range(currency_data.index[-1], periods=forecast_steps + 1, freq='D')[1:]
    currency_forecast_df = pd.DataFrame({'Date': forecast_dates, 'Forecasted_Price': currency_forecast})
    currency_forecast_df.set_index('Date', inplace=True)

    return currency_forecast_df

# Step 6: Visualizing the Dashboard
def create_dashboard(stock_data, currency_data, currency_forecast_df):
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=('Market Price with Anomalies', 'Currency Price & Forecast'))

    # Plot 1: Market Price and Anomalies
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], mode='lines', name='Market Price'), row=1, col=1)
    anomalies = stock_data[stock_data['Anomaly'] == 1]
    fig.add_trace(go.Scatter(x=anomalies.index, y=anomalies['Close'], mode='markers',
                             marker=dict(color='red', size=8), name='Anomalies'), row=1, col=1)

    # Plot 2: Currency Prices and Forecast
    fig.add_trace(go.Scatter(x=currency_data.index, y=currency_data['Close'], mode='lines', name='Currency Price (Actual)'), row=2, col=1)
    fig.add_trace(go.Scatter(x=currency_forecast_df.index, y=currency_forecast_df['Forecasted_Price'],
                             mode='lines', name='Currency Price (Forecast)'), row=2, col=1)

    fig.update_layout(height=800, title_text="Market Data and Currency Risk Dashboard")
    fig.show()

In [ ]:

# Step 7: Flask Web Application for User Input
app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        stock_symbol = request.form['stock_symbol']
        stock_data, currency_data = fetch_market_data(stock_symbol)
        stock_data = detect_anomalies(stock_data)
        currency_forecast_df = currency_forecast(currency_data)

        # Call dashboard function to display results
        create_dashboard(stock_data, currency_data, currency_forecast_df)

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
!pip install flask



In [ ]:
!pip install pyngrok


In [ ]:
# Install Flask and pyngrok
!pip install flask pyngrok

# Import necessary libraries
from flask import Flask, render_template, request, jsonify
from pyngrok import ngrok
import pandas as pd

# Initialize Flask app
app = Flask(__name__)

# Simulate fetching stock and currency data (replace with real data fetch later)
def fetch_market_data(stock_symbol):
    # For example purposes, we generate mock stock data
    stock_data = pd.DataFrame({
        'Date': ['2024-10-01', '2024-10-02', '2024-10-03'],
        'Close': [100, 105, 102]
    })
    # You can also fetch currency data similarly if required
    currency_data = pd.DataFrame({
        'Currency': ['USD', 'EUR', 'JPY'],
        'Value': [1.0, 0.9, 110.0]
    })
    return stock_data, currency_data

# Create the dashboard (you can expand this part to generate visualizations)
@app.route('/', methods=['POST'])
def get_data():
    stock_symbol = request.form['stock_symbol']
    stock_data, currency_data = fetch_market_data(stock_symbol)

    # For simplicity, we return JSON data. You can modify this to return HTML dashboard.
    return jsonify({
        "stock_symbol": stock_symbol,
        "stock_data": stock_data.to_dict(),
        "currency_data": currency_data.to_dict()
    })

# Start ngrok and Flask server
if __name__ == '__main__':
    # Connect ngrok to port 5000
    public_url = ngrok.connect(5000)
    print(f"Flask app is running on {public_url}")

    # Run the Flask app
    app.run(port=5000)


ERROR:pyngrok.process.ngrok:t=2024-10-20T08:09:53+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-10-20T08:09:53+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"
ERROR:pyngrok.process.ngrok:t=2024-10-20T08:09:53+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your aut

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.